<center><h1> Notebook for processing one file</h1>
*Scott Collis, Jonathan Helmus, Kirk North, Tim Lang*
</center>


In [15]:
#needs to be running under pyart35 environment
import pyart
from matplotlib import pyplot as plt
import matplotlib
import numpy as np
from scipy import ndimage, signal, integrate
import time
import copy
import netCDF4
import skfuzzy as fuzz
import datetime
import platform
import fnmatch
import os
%matplotlib inline

In [13]:
my_system = platform.system()
if my_system == 'Darwin':
    top = '/data/sample_sapr_data/sgpstage/sur/'
    soundings_dir = '/data/sample_sapr_data/sgpstage/interp_sonde/'
elif my_system == 'Linux':
    top = '/lcrc/group/earthscience/radar/sgpstage/sur/'
    soundings_dir = '/lcrc/group/earthscience/radar/interp_sonde/'

In [19]:
radar = pyart.io.read('/data/sample_sapr_data/sgpstage/sur/20110520/110635.mdv')
print(radar.fields.keys())


dict_keys(['specific_differential_phase', 'spectrum_width', 'differential_phase', 'velocity', 'reflectivity', 'cross_correlation_ratio', 'differential_reflectivity', 'normalized_coherent_power'])


In [25]:
i_end = 975
radar.range['data']=radar.range['data'][0:i_end]
for key in radar.fields.keys():
    radar.fields[key]['data']= radar.fields[key]['data'][:, 0:i_end]
radar.ngates = i_end

In [20]:
#guess a whole heap of data
radar_start_date = netCDF4.num2date(radar.time['data'][0], radar.time['units'])
print(radar_start_date)
ymd_string = datetime.datetime.strftime(radar_start_date, '%Y%m%d')
hms_string = datetime.datetime.strftime(radar_start_date, '%H%M%S')
print(ymd_string, hms_string)

2011-05-20 11:01:00
20110520 110100


In [22]:
sonde_pattern = datetime.datetime.strftime(radar_start_date,'sgpinterpolatedsondeC1.c1.%Y%m%d.*.cdf')
all_sonde_files = os.listdir(soundings_dir)
sonde_name = fnmatch.filter(all_sonde_files, sonde_pattern)[0]
print(sonde_pattern,sonde_name)

sgpinterpolatedsondeC1.c1.20110520.*.cdf sgpinterpolatedsondeC1.c1.20110520.000000.cdf


In [24]:
interp_sonde = netCDF4.Dataset(os.path.join( soundings_dir, sonde_name))
temperatures = interp_sonde.variables['temp'][:]
times = interp_sonde.variables['time'][:]
heights = interp_sonde.variables['height'][:]
my_profile = pyart.retrieve.fetch_radar_time_profile(interp_sonde, radar)
info_dict = {'long_name': 'Sounding temperature at gate',
             'standard_name' : 'temperature',
             'valid_min' : -100,
             'valid_max' : 100,
             'units' : 'degrees Celsius'}
z_dict, temp_dict = pyart.retrieve.map_profile_to_gates( my_profile['temp'], 
                                         my_profile['height']*1000.0, 
                                         radar)
radar.add_field('sounding_temperature', temp_dict, replace_existing = True)
radar.add_field('height', z_dict, replace_existing = True)
snr = pyart.retrieve.calculate_snr_from_reflectivity(radar)
radar.add_field('SNR', snr, replace_existing = True)
